In [1]:
# autolabel id num


import numpy as np
import sys
sys.path.insert(4, r'C:/Users/Asc-user/Documents/YOLO/darknet')
from darknet_noWinmode import *
import cv2
import glob
import time 
import copy
import os
import signal

def darknet_helper(image, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height),interpolation=cv2.INTER_LINEAR)
    img_height, img_width, _ = image.shape
    width_ratio = img_width/width
    height_ratio = img_height/height
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image, thresh=0.9)
    free_image(darknet_image)

    return detections, width_ratio, height_ratio
       
network, class_names, class_colors = load_network(r"C:/Users/Asc-user/Documents/YOLO/Y562_train/sid_train/yolov4-tiny.cfg", 
                                                  r"C:/Users/Asc-user/Documents/YOLO/Y562_train/sid_train/obj.data", 
                                                  r"C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/shid_num_v1.weight")


path = 'C:/Users/Asc-user/Documents/YOLO/Y562_train/shid_num_2/'
for level_1 in os.listdir(path):
    print(path+level_1)
    if level_1[-4:] == '.png':
        image = cv2.imread(path+level_1)
        size = [image.shape[1],image.shape[0]]
        print(size)
        detections, width_ratio, height_ratio = darknet_helper(image, 352, 160)
        detections = sorted(detections,reverse = False, key = lambda s: s[2])
        print(len(detections))
        for label,confidence, bbox in detections:
                left, top, right, bottom = bbox2points(bbox)
                left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
                top,bottom,left,right = abs(top),abs(bottom),abs(left),abs(right)

                if label =='P':
                    label='10'
                dw = 1./size[0]
                dh = 1./size[1]
                #print(dw,dh)
                x = (left + right)/2.0
                y = (top + bottom)/2.0
                w = right - left
                h = bottom - top
                x = x*dw
                w = w*dw
                y = y*dh
                h = h*dh
                id_path = path+level_1[:-4]
                write = open(id_path+'.txt', 'a')
                print(label ,x,y,w,h,file=write)

                write.close() 
print('done')

C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/0723
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/0724
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/0725
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/classes.txt
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_000150_id.png
[247, 243]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_000300_id.png
[221, 273]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_000450_id.png
[216, 249]
5
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_000600_id.png
[197, 250]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_000900_id.png
[363, 135]
1
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_001050_id.png
[332, 128]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_001200_id.png
[339, 128]
3
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_001350_id.png
[332, 128]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid2/moving_001500_id.png
[339, 128]
3
C:/Users/Asc-user/Documents/YO

In [5]:
import numpy as np
import sys
import math
sys.path.insert(4, r'C:/Users/Asc-user/Documents/YOLO/darknet')
from darknet_noWinmode import *
import cv2
import glob
import time 
import copy
import os
import signal

def rotate_image(image, angle):
    # 進行圖片旋轉
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    return rotated_image

def rotate_bbox(bbox, angle, image_width, image_height):
    # 將bounding box旋轉回原圖座標
    x, y, w, h = bbox
    cx, cy = x + w / 2, y + h / 2
    new_cx = cx * np.cos(angle) - cy * np.sin(angle)
    new_cy = cx * np.sin(angle) + cy * np.cos(angle)
    new_w = w * abs(np.cos(angle)) + h * abs(np.sin(angle))
    new_h = w * abs(np.sin(angle)) + h * abs(np.cos(angle))
    
    # 根據旋轉角度調整座標，考慮圖片的長寬不是正方形的情況
    new_x = new_cx - new_w / 2
    new_y = new_cy - new_h / 2
    new_x = max(0, min(new_x, image_width - new_w))
    new_y = max(0, min(new_y, image_height - new_h))
    new_w = max(0, min(new_w, image_width - new_x))
    new_h = max(0, min(new_h, image_height - new_y))
    
    return [new_x, new_y, new_w, new_h]

def darknet_helper(image, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height),interpolation=cv2.INTER_LINEAR)
    img_height, img_width, _ = image.shape
    width_ratio = img_width/width
    height_ratio = img_height/height
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image, thresh=0.9)
    free_image(darknet_image)

    return detections, width_ratio, height_ratio

network, class_names, class_colors = load_network("C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/CS_ID/yolov4-tiny.cfg", 
                                                  "C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/CS_ID/obj.data", 
                                                  "C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/cs_id_v12.weights")

path = 'C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/'
for level_1 in os.listdir(path):
    print(path + level_1)
    if level_1[-4:] == '.png':
        image = cv2.imread(path + level_1)
        size = [image.shape[1], image.shape[0]]
        print(size)
        
        # 原始圖片辨識
        detections, width_ratio, height_ratio = darknet_helper(image, 352, 160)
        detections = sorted(detections, reverse=False, key=lambda s: s[2])
        print(len(detections))
        
        # 左轉30度辨識
        image_rotated_left = rotate_image(image, 30)
        detections_left, _, _ = darknet_helper(image_rotated_left, 352, 160)
        detections_left = sorted(detections_left, reverse=False, key=lambda s: s[2])
        
        # 右轉30度辨識
        image_rotated_right = rotate_image(image, -30)
        detections_right, _, _ = darknet_helper(image_rotated_right, 352, 160)
        detections_right = sorted(detections_right, reverse=False, key=lambda s: s[2])
        
        # 找出辨識物件數量最多且信心指數最高的結果
        if len(detections_left) >= len(detections) and len(detections_left) >= len(detections_right):
            best_detections = detections_left
            best_image = image_rotated_left
            angle = 30
        elif len(detections_right) >= len(detections) and len(detections_right) >= len(detections_left):
            best_detections = detections_right
            best_image = image_rotated_right
            angle = -30
        else:
            best_detections = detections
            best_image = image
            angle = 0
        
        for label, confidence, bbox in best_detections:
            left, top, right, bottom = bbox2points(bbox)
            left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
            top, bottom, left, right = abs(top), abs(bottom), abs(left), abs(right)

            if label =='P':
                label='10'
            dw = 1./size[0]
            dh = 1./size[1]
            #print(dw,dh)
            x = (left + right)/2.0
            y = (top + bottom)/2.0
            w = right - left
            h = bottom - top
            
            # 將座標旋轉回原圖座標
            rotated_bbox = rotate_bbox([x, y, w, h], math.radians(-angle), image.shape[1], image.shape[0])
            x, y, w, h = rotated_bbox

            x = x * dw
            w = w * dw
            y = y * dh
            h = h * dh
            id_path = path + level_1[:-4]
            write = open(id_path + '.txt', 'a')
            print(label, x, y, w, h, file=write)

            write.close() 
print('done')


C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_11-59-57_id.png
[240, 263]
5
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-02-07_id.png
[291, 166]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-23-09_id.png
[179, 222]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-25-15_id.png
[196, 197]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-26-24_id.png
[271, 152]
3
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-27-36_id.png
[271, 128]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-28-28_id.png
[277, 138]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-29-26_id.png
[308, 184]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-29-54_id.png
[222, 256]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-30-41_id.png
[280, 141]
5
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-31-27_id.png
[301, 225]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-32-12_id.png
[252,

In [6]:
import numpy as np
import sys
import math
sys.path.insert(4, r'C:/Users/Asc-user/Documents/YOLO/darknet')
from darknet_noWinmode import *
import cv2
import glob
import time 
import copy
import os
import signal

def rotate_image(image, angle):
    # 進行圖片旋轉
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    return rotated_image

def darknet_helper(image, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height),interpolation=cv2.INTER_LINEAR)
    img_height, img_width, _ = image.shape
    width_ratio = img_width/width
    height_ratio = img_height/height
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image, thresh=0.9)
    free_image(darknet_image)

    return detections, width_ratio, height_ratio

network, class_names, class_colors = load_network("C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/CS_ID/yolov4-tiny.cfg", 
                                                  "C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/CS_ID/obj.data", 
                                                  "C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/cs_id_v12.weights")

path = 'C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/'
for level_1 in os.listdir(path):
    print(path + level_1)
    if level_1[-4:] == '.png':
        image = cv2.imread(path + level_1)
        size = [image.shape[1], image.shape[0]]
        print(size)
        
        # 原始圖片辨識
        detections, width_ratio, height_ratio = darknet_helper(image, 352, 160)
        detections = sorted(detections, reverse=False, key=lambda s: s[2])
        print(len(detections))
        
        # 左轉30度辨識
        image_rotated_left = rotate_image(image, 30)
        detections_left, _, _ = darknet_helper(image_rotated_left, 352, 160)
        detections_left = sorted(detections_left, reverse=False, key=lambda s: s[2])
        
        # 右轉30度辨識
        image_rotated_right = rotate_image(image, -30)
        detections_right, _, _ = darknet_helper(image_rotated_right, 352, 160)
        detections_right = sorted(detections_right, reverse=False, key=lambda s: s[2])
        
        # 找出辨識物件數量最多且信心指數最高的結果
        if len(detections_left) >= len(detections) and len(detections_left) >= len(detections_right):
            best_detections = detections_left
            best_image = image_rotated_left
            angle = 30
        elif len(detections_right) >= len(detections) and len(detections_right) >= len(detections_left):
            best_detections = detections_right
            best_image = image_rotated_right
            angle = -30
        else:
            best_detections = detections
            best_image = image
            angle = 0
        
        # 將結果儲存並修改txt檔案名稱
        id_path = path + level_1[:-4] + '_result.txt'
        write = open(id_path, 'w')
        for label, confidence, bbox in best_detections:
            left, top, right, bottom = bbox2points(bbox)
            left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
            top, bottom, left, right = abs(top), abs(bottom), abs(left), abs(right)

            if label =='P':
                label='10'
            dw = 1./size[0]
            dh = 1./size[1]
            #print(dw,dh)
            x = (left + right)/2.0
            y = (top + bottom)/2.0
            w = right - left
            h = bottom - top

            x = x * dw
            w = w * dw
            y = y * dh
            h = h * dh

            print(label, x, y, w, h, file=write)

        write.close() 
print('done')


C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_11-59-57_id.png
[240, 263]
5
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-02-07_id.png
[291, 166]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-23-09_id.png
[179, 222]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-25-15_id.png
[196, 197]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-26-24_id.png
[271, 152]
3
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-27-36_id.png
[271, 128]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-28-28_id.png
[277, 138]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-29-26_id.png
[308, 184]
6
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-29-54_id.png
[222, 256]
2
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-30-41_id.png
[280, 141]
5
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-31-27_id.png
[301, 225]
7
C:/Users/Asc-user/Documents/YOLO/Y562_train/sid/000000_13-32-12_id.png
[252,